In [2]:
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import text_extract as text
import json
import os
import re

In [13]:
JSON_PATH = "./settings.json"

with open(JSON_PATH) as f:
    settings = json.load(f)

file_settings = settings["file"]
file_names = text.data.get_file_names(file_settings)

image_arr = []
h_crop_point_arr = []
v_crop_point_arr = []
h_max_size = 0
v_max_size = 0

for name in file_names:
    path = file_settings["path"]
    gray_img = text.data.read_image(path, name)

    rotate_img = text.extract.set_textbox_horizontally(gray_img, settings)
    textbox_img = text.extract.extract_textbox(rotate_img, settings)
    enhanced_img = text.extract.enhance_textbox(textbox_img, settings)

    char_crop_settings = settings["cropPoints"]
    [h_crop_points, h_max] = text.data.get_char_info(enhanced_img, char_crop_settings, mode="h")
    [v_crop_points, v_max] = text.data.get_char_info(enhanced_img, char_crop_settings, mode="v")

    image_arr.append(enhanced_img)

    h_crop_point_arr.append(h_crop_points)
    v_crop_point_arr.append(v_crop_points)

    h_max_size = h_max if h_max > h_max_size else h_max_size
    v_max_size = v_max if v_max > v_max_size else v_max_size

(11, 20)

In [ ]:
# 헷갈릴 글자 대상들
# 0 Γ
# 8 Δ
# 5 Σ
# $ -> § or 직접 폰트 수정
#  (공백) Ч
# ` §

# 절대로 헷갈리지 않을 만한 문자들
# 그리스어
# Γ Δ Ξ Σ